Wav2Vec 2.0 WER on LibriSpeech clean-test with added noise and LibriSpeech other-test

In [1]:
pip install datasets jiwer transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 30.0 MB/s 
     |████████████████████████████████| 5.3 MB 64.3 MB/s 
     |████████████████████████████████| 115 kB 46.5 MB/s 
     |████████████████████████████████| 163 kB 62.0 MB/s 
     |████████████████████████████████| 212 kB 73.3 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
     |████████████████████████████████| 1.4 MB 49.4 MB/s 
     |████████████████████████████████| 2.2 MB 47.0 MB/s 
     |████████████████████████████████| 7.6 MB 53.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset
from jiwer import wer
import librosa
import nltk
import os
import tarfile
import torch
import urllib.request
import soundfile as sf
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor, AutoModelForCTC, AutoProcessor
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# set paths
datasets_path = os.path.join(os.getcwd(), 'datasets') 
# create folders if they do not already exist
if not os.path.exists(datasets_path): os.makedirs(datasets_path)
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def download_and_extract_dataset_from_url(url: str, datasets_path: str = datasets_path):
    """
    downloads and extracts dataset from url into datasets_path/
    """
    temp = os.path.join(datasets_path, url.split('/')[-1])
    urllib.request.urlretrieve(url, temp)
    file = tarfile.open(temp)
    file.extractall(datasets_path)
    file.close()
    os.remove(temp)

In [5]:
download_and_extract_dataset_from_url('https://www.openslr.org/resources/12/test-clean.tar.gz')
download_and_extract_dataset_from_url('https://www.openslr.org/resources/12/test-other.tar.gz')

In [6]:
# load extracted lr data as dataset
librispeech_clean = load_dataset("datasets/LibriSpeech/test-clean", "clean", split='train')
librispeech_other = load_dataset("datasets/LibriSpeech/test-other", "other", split='train')

Resolving data files:   0%|          | 0/2707 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/87 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/test-clean-384505b0824d9773/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


Resolving data files:   0%|          | 0/3029 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/90 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/test-other-0ea95b7e4a27cbdf/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


In [7]:
def map_to_ground_truth(batch):
    """
    inserts ground truth in dataset
    """
    transcription_file_path = batch['audio']['path'][:-10] + '.trans.txt'
    f = open(transcription_file_path, 'r')
    lines= str.splitlines(f.read())
    txt=lines[int(batch['audio']['path'][-7:-5])].split(' ', 1)[1]
    batch['txt'] = txt
    return batch

In [8]:
librispeech_clean = librispeech_clean.map(map_to_ground_truth)
librispeech_other = librispeech_other.map(map_to_ground_truth)

  0%|          | 0/2620 [00:00<?, ?ex/s]

  0%|          | 0/2939 [00:00<?, ?ex/s]

In [9]:
def load_wav2vec_model(hf_path: str):
    """
    load and return wav2vec tokenizer and model from huggingface
    """
    tokenizer = Wav2Vec2Tokenizer.from_pretrained(hf_path)
    model = Wav2Vec2ForCTC.from_pretrained(hf_path).to(device)
    return tokenizer, model

In [10]:
tokenizer, model = load_wav2vec_model("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def map_to_pred(batch):
    """
    predicts transcription
    """
    #tokenize
    input_values = tokenizer(batch["audio"]["array"], return_tensors="pt").input_values
    #take logits
    logits = model(input_values.to(device)).logits
    #take argmax (find most probable word id)
    predicted_ids = torch.argmax(logits, dim=-1)
    #get the words from the predicted word ids
    transcription = tokenizer.decode(predicted_ids[0])
    batch["transcription"] = transcription
    return batch

In [12]:
clean_result = librispeech_clean.map(map_to_pred)
other_result = librispeech_other.map(map_to_pred)

  0%|          | 0/2620 [00:00<?, ?ex/s]

  0%|          | 0/2939 [00:00<?, ?ex/s]

In [17]:
def calculate_wer(text, transcription, decimal=1):
  return round(100 * wer(text, transcription), decimal)

In [21]:
print('WER with wav2vec2-base-960h on lr-test-clean:', calculate_wer(clean_result["txt"], clean_result["transcription"]), '%.')
print('WER with wav2vec2-base-960h on lr-test-other:', calculate_wer(other_result["txt"], other_result["transcription"]), '%.')

WER with wav2vec2-base-960h on lr-test-clean: 3.4 %.
WER with wav2vec2-base-960h on lr-test-other: 9.3 %.
